In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Simulated CMB skies

It is often very useful to make simulations of the CMB sky, to gain physical intuition, to test code, to calibrate estimators, etc.  For the primary anisotropies this is very straightforward, as the CMB is essentially a Gaussian random field (with a few wrinkles we'll discuss later).  There are two, basic kinds of mock skies people often make: (1) a "flat" sky, usually a relatively small patch and (2) a full sky.  The former is incredibly useful for trying things out and is very computationally efficient as it uses FFTs.  The latter is often used as an analysis or idea gets more refined and you want to be able to apply it with all of the bells and whistles.

## Primary (temperature) anisotropies ##

To generate the primary anisotropy spectrum we start with an angular power spectrum, $C_\ell$.  At this point you can read in a pre-computed spectrum or generate one yourself.  We'll do the latter, using ...

### CLASS Boltzmann code ###

Just a reminder from the earlier notebook, there are two "commonly used" codes which evolve the linearized perturbation equations for cosmology.  These are highly useful tools for working in the CMB or large-scale structure, so here I introduce one of them: the [Cosmic Linear Anisotropy Solving System](http://class-code.net/).  This code is very well documented and maintained, and a public version is avaialble [at this GitHub repo](https://github.com/lesgourg/class_public), making it easy to keep up to date (or to contribute!).

If you clone the repository, e.g. with
`git clone https://github.com/lesgourg/class_public class`
then you can just go into the class directory and "make" the code.  I've never had a problem with this on any machine that a few edits to the Makefile didn't fix.  If you want to install the Python interface just go into the Python directory and use `python setup.py install`.  On the datahub you can install as root so you don't need the `--user`, otherwise you can use your own system.  There is also a `Conda` version if you use an Anaconda python distribution.

In [ ]:
# Import the Class class.
from classy import Class
# Define the cosmology (what is not specified will be set to CLASS default parameters).
# We'll use the "Planck 2018 cosmology" (plus BAO), defined in the right-hand column
# of Table 6 in https://arxiv.org/pdf/1807.06205.pdf
# We'll ask for CMB temperature anisotropy spectra, lensed spectra and matter P(k):
params = {
    'output': 'tCl lCl mPk',
    'l_max_scalars': 3000,
    'P_k_max_h/Mpc': 20.,
    'lensing': 'yes',
    'A_s': np.exp(3.040)*1e-10,
    'n_s': 0.96824,
    'h': 0.6770,
    'N_ur': 2.0328,
    'N_ncdm': 1,
    'm_ncdm': 0.06,
    'tau_reio': 0.0568,
    'omega_b': 0.022447,
    'omega_cdm': 0.11923}  
cosmo = Class()
cosmo.set(params)
cosmo.compute()
# Get the unlensed CMB spectrum
cls = cosmo.raw_cl(2500)

### Flat sky ###

Let's first do the "flat sky" case.  This is valid for small patched of the sky, and the great advantage is that we can replace spherical harmonics (eigenfunctions of the Laplacian on the sphere) by Fourier transforms (with $\exp[ikx]$ being eigenfunctions of the Laplacian on the plane).  Here we take a small patch of sky ($\sim 10^\circ \times 10^\circ$) and replace $\ell=|k|$.

Generating a Gaussian random field from a power spectrum can be done in one of two ways.  In the first we generate Gaussian random amplitudes and phases for each complex $a(\vec{k})$ such that $\langle |a(\vec{k})|^2\rangle=C_\ell$.  The amplitude is drawn from a Rayleigh distribution and the phases from a uniform distribution.  This just involves some bookkeeping with $\sqrt{2}$ and the like (and SciPy has routines for everything).  An alternative is to generate a white noise field of unit variance and then convolve it with $\sqrt{C_\ell}$.  We'll use the alternative method ...

In [ ]:
Lside = 10.0 * np.pi/180.  # Map is 10 degrees on a side.
Npix  = 128
kv    = 2*np.pi/Lside * Npix*np.fft.fftfreq(Npix)
kx,ky = np.meshgrid(kv,kv)
kk    = np.sqrt( kx**2+ky**2 ).clip(0,2500)
ell   = kk.astype('int')
#
temp  = np.random.normal(0,1,(Npix,Npix))
temp  = np.fft.fft2(temp) * np.sqrt(cls['tt'][ell])
temp  = np.real( np.fft.ifft2(temp) ) / (Lside/Npix)
#
print("CMB map has {:12.4e}<T<{:12.4e}".format(np.min(temp),np.max(temp)))

In [ ]:
LL = Lside/2 / (np.pi/180.)
plt.imshow(temp.T,origin='lower',extent=[-LL,LL,-LL,LL])

## Foregrounds ##

Now we can add foregrounds.  Foregrounds can be classified as galactic and extragalactic, or as (unresolved) point sources or extended sources.  For the extragalactic sources we can think of them as "Poisson" or "clustered".

### Point sources ###

Let us do the simplest case of Poisson (unclustered) point sources.  For these sources we can draw their locations at random, and then we just need to specify their luminosity function, for example a double power law or a Schechter function, or a single power-law with cutoffs etc.  In order to sample from the luminosity function efficiently, we need the inverse of the cumulative luminosity function.  This is easiest for a power-law, so let's use that:
$$
  \frac{dN}{dS}\propto S^{\alpha} \quad\Rightarrow\quad
  x=\int dN\propto S^{\alpha+1}-S_{\rm min}^{\alpha+1} \quad\Rightarrow\quad
  S=\left[x(S_{\rm max}^{\alpha+1}-S_{\rm min}^{\alpha+1})+S_{\rm min}^{\alpha+1}\right]^{1/(\alpha+1)}
$$
if $x$ is uniform in $[0,1)$.

While we won't need it directly, a useful result is that sources of flux $S$ contribute
$$
  C_\ell = \int_0^{S_{\rm cut}} S^2\frac{dN}{dS}\,dS + w_\ell\left( \int_0^{S_{\rm cut}} S\frac{dN}{dS}\,dS \right)^2
$$
assuming all sources of $S>S_{\rm cut}$ are removed or masked and that the remainder have angular power spectrum $w_\ell$.  The first term is the shot-noise term, which is usually dominant on small scales (high $\ell$).

In [ ]:
Nsrc = 1000
xx   = np.random.uniform(size=Nsrc)
yy   = np.random.uniform(size=Nsrc)
ap1  = -1.5 + 1
Lmin = 1e-5
Lmax = 1e-3
flux =(np.random.uniform(size=Nsrc)*(Lmax**ap1-Lmin**ap1)+Lmin**ap1)**(1./ap1)
ps,x,y=np.histogram2d(xx,yy,bins=[Npix,Npix],range=[[0,1],[0,1]],weights=flux)
#
print("Assigned {:d} sources, total flux {:12.4e} ({:12.4e})".\
      format(flux.size,np.sum(ps),np.sum(flux)))

In [ ]:
plt.imshow(ps.T,origin='lower',extent=[-LL,LL,-LL,LL])

### Extended source example: SZ clusters ###

To model extended sources the easiest thing to do is to take a point source map and then convolve it with a profile function using FFTs.  This is very straightforward using FFTs as long as all of the sources have the same "size", but this is not in general true.  If it *is* true, then we can either get an analytic profile function to multiply our map by in k-space before inverting it *or* we can lay down a profile, FT it and then multiply our map FT by it before inverting.

We have several choices for cluster profiles.  We could use the projection of an [NFW](https://en.wikipedia.org/wiki/Navarro%E2%80%93Frenk%E2%80%93White_profile) profile, but in general the gas doesn't trace the DM.  We could assume the gas is in [hydrostatic equilibrium with a polytropic equation-of-state in an NFW potential](https://arxiv.org/abs/astro-ph/0106151).  This allows us to derive an analytic gas profile which is close to what's observed.  We could take a "universal gas profile", or one of its modifications.  Or we could use an X-ray inspired model: [the $\beta$-profile](http://adsabs.harvard.edu/abs/1984ApJ...276...38J), which is a very old fitting function which goes back to the early days of X-ray cluster astronomy.

We'll choose all clusters to have the same luminosity and use a simple Gaussian profile of fixed size!

In [ ]:
Nclus= 50
xx   = np.random.uniform(size=Nclus)
yy   = np.random.uniform(size=Nclus)
c,x,y= np.histogram2d(xx,yy,bins=[Npix,Npix],range=[[0,1],[0,1]])
c   *= 1e-4
print("Map range is {:e} to {:e}".format(np.min(c),np.max(c)))
# Set up the plumbing for a Gaussian profile:
kv   = 2*np.pi/Lside * Npix*np.fft.fftfreq(Npix)
kx,ky= np.meshgrid(kv,kv)
k2   = kx**2+ky**2
# Let's make the clusters "huge" so we can see easily that they
# are extended, by eye.  This isn't very realistic ...
sig2 = (15./60. * np.pi/180)**2
gauss= np.exp(-0.5*k2*sig2)
# FT the map, multiply by the Gaussian and invert the FT.
c = np.real( np.fft.ifft2( np.fft.fft2(c)*gauss ) )
# and let's take a look at it.
plt.imshow(c.T,origin='lower',extent=[-LL,LL,-LL,LL])

### Lensing ###

At this point you could add lensing.  In the simplest incarnation you could generate a Gaussian deflection field from $C_\ell^{\phi\phi}$ (i.e. ```cls[pp]```).  You can compute gradients of this field simply by Fourier transforming and multiplying by $k_x$ or $k_y$.  Now you have a deflection everywhere, you just have to rebin your map to the "new" positions.  There are pixel issues to deal with in this rebinning, but it's not difficult in principle it's just a matter of careful bookkeeping.  We'll skip this step for now, but you might be interested in the [QuickLens](https://github.com/dhanson/quicklens) software package.

### Adding beam and noise ###

Add in noise and beam.

In [ ]:
# We'll do a very simple noise model, just Gaussian, white noise
# of fixed amplitude.  In principle you would "run" your observation
# on the signal map, add in noise in the time stream using a
# simulated detector, then put it back onto the map using the same
# map making algorithm as the experiment.
noise = np.random.normal(0,1e-4,size=(Npix,Npix))
# We can do a Gaussian beam (or other beam shape) the same
# way we did a cluster profile -- using a "window function"
# for the beam.
tot  = temp + ps + c + noise
#
kv   = 2*np.pi/Lside * Npix*np.fft.fftfreq(Npix)
kx,ky= np.meshgrid(kv,kv)
k2   = kx**2+ky**2
sig2 = (5./60. * np.pi/180)**2
beam = np.exp(-0.5*k2*sig2)
tot  = np.real( np.fft.ifft2( np.fft.fft2(tot)*beam ) )
# and let's take a look at it.
plt.imshow(tot.T,origin='lower',extent=[-LL,LL,-LL,LL])

The next step would be the analyze this map ... but for now let us turn to how we would do this on the full sky!

# Full-sky #

If you are familiar with the Healpix routines, then doing the same thing on the full sky is relatively straightforward -- in fact somewhat easier!  You make a random sky map, do a spherical harmonic transform and multiply by $\sqrt{C_\ell}$, then invert the harmonic transform.  This is a common enough procedure that it can be done in one step using ```hp.synfast(cls,nside)```... a routine which also allows you to generate polarized maps (though we won't).

## Healpix ##

While there are several ways of pixelizing the sphere (i.e. putting scalar or tensor data onto the sphere in a discrete way) the cosmology community almost uniformly uses the [Healpix](https://healpix.jpl.nasa.gov/) scheme.  Healpix is built around 12 base pixels (each about 1 steradian in area) which are then subdivided into 4, each of those in 4 again and so on.  You can see pictures in the documentation link earlier.  There are two advantages of Healpix for our purposes:

1. Each pixel is approximately equal area, so integrals over the sphere become sums over pixels in a simple way.

2. Pixels lie in iso-latitude rings, i.e. groups have the same $\theta$ in spherical coordinates.  This means in the spherical harmonic transform, the $\phi$ integral becomes an FFT (at fixed $\theta$), and the $\theta$ integral can be done using recurrence relations among the $P_\ell^m$.

We'll use the `Healpy` package for manipulating Healpix maps (more on other packages below).  You can install `Healpy` using pip, e.g. from within a notebook
```
!pip install healpy
```
or from the terminal just pip install.
Beware that the Conda version uses obsolete libraries and this can cause "issues".

Assuming you have installed ```healpy``` ...

In [ ]:
import healpy as hp
temp = hp.synfast(cls['tt'],nside=64,new=True)
hp.mollview(temp)

The `synfast` routine can also return the $a_{\ell m}$ of the generated sky at the same time (see the documentation).

Adding point sources or making extended sources follows the same procedure as above.  You need to generate random $\theta,\phi$ on the sky (draw uniform in $\phi$ and in $\cos\theta$), convert them to pixel numbers using ```hp.ang2pix(nside,theta,phi)```, then histogram.  Beam smoothing or profile convolution can be done using e.g. ```hp.smoothing```.

## Beyond Gaussian skies ##

Of course we could demand more realism from our simulations.  First off, we could imagine generating the point sources, clusters, etc. self-consistently with the low-redshift structure which is causing our lensing.  We could also include the fact that at lower redshift structure goes non-linear and is non-Gaussian.  We could include the clustering of our point sources or clusters, etc.

Perhaps the easiest way to include all of these effects self-consistently is to build upon a simulation (either N-body, hydro or based on perturbation theory) with recipes for including the various relevant effects.  One, publicly available, example is the [WebSky mocks](https://mocks.cita.utoronto.ca/index.php/WebSky_Extragalactic_CMB_Mocks) which are being used as the extra-galactic "model" by several on-going or planned experiments.

Foregrounds and other components are frequently added using the [PySM](https://github.com/bthorne93/PySM_public) package.  You can use the [TOAST](https://theodorekisner.com/software/toast/) tools to "run a mission" over a simulated sky to produce timelines, then turn those timelines into maps, masks, etc.  Many (though not all) of the most useful packages have been brought together in the [SO GitHub](https://github.com/simonsobs) or the [CMB-S4 GitHub](https://github.com/CMB-S4).